In [6]:
# Import necessary libraries
import numpy as np
import pandas as pd
import requests, zipfile
import datetime as dt

from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

The [rain areas section](https://www.nea.gov.sg/weather/rain-areas) of the NEA website of Singapore provides radar scans at 5-minute intervals. Here is an example of the rain areas at a 240km radius around Singapore at 2024-08-24 21:30:00 :

![rain areas](figs/dpsri_240km_2024082421300000dBR.dpsri.png)

With a little guesswork, it is possible to figure out the path to which NEA stores the raw images. The code and the widgets below will download images at 70km and 240km distances and store them in a zip file.

Note that not **all** timestamps are available for download.

In [7]:
# Function to adjust the start time to the nearest 5-minute mark
def adjust_to_five_minute_mark(dt_obj):
    # Round up to the nearest 5 minutes
    if dt_obj.minute % 5 != 0 or dt_obj.second != 0:
        delta_minutes = 5 - (dt_obj.minute % 5)
        dt_obj += dt.timedelta(minutes=delta_minutes)
        dt_obj = dt_obj.replace(second=0, microsecond=0)
    return dt_obj

# Function to generate the sequence of strings
def generate_sequence(start_date, start_time, end_date, end_time):
    # Parse start and end datetime objects
    start_datetime = dt.datetime.combine(start_date, dt.datetime.strptime(start_time, '%H:%M:%S').time())
    end_datetime = dt.datetime.combine(end_date, dt.datetime.strptime(end_time, '%H:%M:%S').time())

    # Adjust the start time to the nearest 5-minute mark
    start_datetime = adjust_to_five_minute_mark(start_datetime)

    # Generate the timestamps with 5-minute intervals
    timestamps = pd.date_range(start=start_datetime, end=end_datetime, freq='5min')

    # Convert each timestamp to the desired string format
    formatted_strings = [timestamp.strftime('%Y%m%d%H%M') + '0000' for timestamp in timestamps]

    return formatted_strings


In [8]:
url70 = ['https://www.nea.gov.sg/docs/default-source/rain-area/dpsri_70km_', 
        '202112270530', 'dBR.dpsri.png']
url240 = ['https://www.nea.gov.sg/docs/default-source/rain-area-240km/dpsri_240km_', 
        '202112250810','dBR.dpsri.png']

# function to run through the list of timestamps, download the images and 
# save them to a zipfile.
def get_images(timestamp_list, outzipfile):
    for url_list in (url70, url240):
        for datetime_str in timestamp_list:
            full_url = ''.join([url_list[0], datetime_str, url_list[2]])
            try: 
                r1 = requests.get(full_url)
                if r1.status_code == 200:
                    print(f'success for {datetime_str}.')
                    fname = Path(full_url).stem + '.png'
                    with open(fname, 'wb') as f:
                        f.write(r1.content)
                        print(f'{fname} written..')
                    
                    with zipfile.ZipFile(file=outzipfile, mode='a', compression=zipfile.ZIP_DEFLATED) as ff:
                        ff.write(fname)
                        Path(fname).unlink()
                else:
                    print(f'failure for {full_url}!')
            except Exception as e:
                print(f"An error occurred for {datetime_str}: {e}")
    print(f"zipfile {outzipfile} written.")

In [9]:
# Date picker widgets for selecting start and end dates
start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
start_time_picker = widgets.Text(
    value='11:00:00',
    placeholder='HH:MM:SS',
    description='Start Time',
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False
)
end_time_picker = widgets.Text(
    value='11:10:00',
    placeholder='HH:MM:SS',
    description='End Time',
    disabled=False
)

# Display widgets
#display(start_date_picker, start_time_picker, end_date_picker, end_time_picker)
display(widgets.VBox( [widgets.HBox([start_date_picker, start_time_picker]),
                       widgets.HBox([end_date_picker, end_time_picker])]))

# Button to generate the sequence
generate_button = widgets.Button(description="Download radar images",
                                 layout=widgets.Layout(width='200px', height='50px'))

# Output area to display results
output = widgets.Output()

# Button click event handler
def on_button_click(b):
    with output:
        output.clear_output()
        # Generate and display the sequence of strings
        sequence = generate_sequence(start_date_picker.value, start_time_picker.value, end_date_picker.value, end_time_picker.value)
        get_images(sequence, 'outfile.zip')
        #for item in sequence:
        #    print(item)

# Assign the click event handler to the button
generate_button.on_click(on_button_click)

# Display the button and output area
display(generate_button, output)

Button(description='Download radar images', layout=Layout(height='50px', width='200px'), style=ButtonStyle())

Output()